This example is actually identical to Example3 except we push some of the functionality into the Code.

AND we start to develop code to organize and keep track of the NLP!!


In [1]:
using NLOptControl
using Polynomials
using Plots
using FastGaussQuadrature
using Parameters
pyplot()

"""
ps, nlp = initialize_NLP(numStates=1,numControls=3);
--------------------------------------------------------------------------------------\n
Author: Huckleberry Febbo, Graduate Student, University of Michigan
Date Create: 1/1/2017, Last Modified: 1/2/2017 \n
Citations: \n
----------\n
Influenced by: S. Hughes.  steven.p.hughes@nasa.gov
Source: DecisionVector.m [located here](https://sourceforge.net/p/gmat/git/ci/264a12acad195e6a2467cfdc68abdcee801f73fc/tree/prototype/OptimalControl/LowThrust/@DecisionVector/)
--------------------------------------------------------------------------\n
"""

# should only call this once
function initialize_NLP2(args...;numStates::Int64=0,numControls::Int64=0,Ni::Int64=10,Nck::Array{Int64,1}=4*ones(Int64,Ni,), kwargs...)
    # validate input
    if length(Nck) != Ni
        error("length(Nck) != Ni");
    end
    for int in 1:Ni
        if (Nck[int]<0)
            error("Nck must be > 0");
        end
    end
    if  Ni <= 0
        error("Ni must be > 0");
    end
    if  numStates <= 0
        error("numStates must be > 0","\n",
                "default value = 0","\n",
              );
    end
    if  numControls <= 0
        error("eventually numControls must be > 0","\n",
              "default value = 0","\n",
              );
    end

    # initialize node data TODO -> eventually make different PS methods available

    taus_and_weights = [gaussradau(Nck[int]) for int in 1:Ni];
    τ = [taus_and_weights[int][1] for int in 1:Ni];
    ω = [taus_and_weights[int][2] for int in 1:Ni];

    # calculate general properties
    numStatePoints = [(Nck[int]+1) for int in 1:Ni]; # number of states dvs (decisionVector) for each interval and each state
    numControlPoints = [Nck[int] for int in 1:Ni];   # number of control dvs for each interval and each control

    # calculate length of vectors
    lengthStateVector = sum([numStatePoints[int]*numStates for int in 1:Ni]);
    lengthControlVector = sum([numControlPoints[int]*numControls for int in 1:Ni]);
    lengthDecVector = lengthStateVector + lengthControlVector + 2; # + 2 is for t0 and tf

    if length(args) == 0
        stateVector = zeros(lengthStateVector,);
        controlVector = zeros(lengthControlVector,);
        decisionVector = zeros(lengthDecVector,);
        t0 = 0.0;
        tf = 0.0;
    else
        # validate optional input
        if  length(args[1]) != lengthStateVector
            error(string("length of stateVector must be = ",lengthStateVector));
        end
        if  length(args[2]) != lengthControlVector
            error(string("length of controlVector must be = ",lengthControlVector));
        end
        if  length(args[3]) != lengthDecVector
            error(string("length of decisionVector must be = ",lengthDecVector));
        end
        if  length(args[4]) != 1
            error(string("length of t0 must be = 1"));
        end
        if  length(args[5]) != 1
            error(string("length of tf must be = 1"));
        end
        stateVector = args[1];
        controlVector = args[2];
        decisionVector = args[3];
        t0 = args[4];
        tf = args[5];
    end

    # determine indecies within overall decision vector of all variables (i.e. decisionVector)
    stateStartIdx = 1;
    stateMidddleIdx = cumsum
    stateStopIdx = stateStartIdx + lengthStateVector -1; # -1 because we start on 1
    controlStartIdx = stateStopIdx + 1;
    controlMiddleIdx = cumsum
    controlStopIdx = controlStartIdx + lengthControlVector -1; # -1 because we start on 1
    timeStartIdx = controlStopIdx + 1;
    timeStopIdx = timeStartIdx + 1;

    if 1==2 #TODO eventually make print_level an option
      print(string("lengthStateVector = ", lengthStateVector),"\n")
      print(string("lengthControlVector = ", lengthControlVector),"\n")

      print(string("stateStartIdx = ", stateStartIdx),"\n")
      print(string("stateStopIdx = ", stateStopIdx),"\n")
      print(string("controlStartIdx = ", controlStartIdx),"\n")
      print(string("controlStopIdx = ", controlStopIdx),"\n")
      print(string("timeStartIdx = ", timeStartIdx),"\n")
      print(string("timeStopIdx = ", timeStopIdx),"\n")
    end

    # check indecies
    if timeStopIdx != lengthDecVector
      error(string("\n",
                    "-------------------------------------", "\n",
                    "There is an error with the indecies!!", "\n",
                    "-------------------------------------", "\n",
                    "The following variables should be equal:", "\n",
                    "timeStopIdx = ",timeStopIdx,"\n",
                    "lengthDecVector = ",lengthDecVector,"\n"
                    )
            )
    end

    # each row contains Xi in the stateMatrix where the size = Nck[int]XnumStates
    stateMatrix=[zeros((Nck[int]+1),numStates) for int in 1:Ni];
    controlMatrix=[zeros(Nck[int],numStates) for int in 1:Ni];
    if true
      print(string("typeof(Nck) = ",typeof(Nck),"\n"))
      print(string("typeof(Ni) = ",typeof(Ni),"\n"))
      print(string("typeof(τ) = ",typeof(τ),"\n"))
      print(string("typeof(ω) = ",typeof(ω),"\n"))
      print(string("typeof(t0) = ",typeof(t0),"\n"))
      print(string("typeof(tf) = ",typeof(tf),"\n"))
      print(string("typeof(stateMatrix) = ",typeof(stateMatrix),"\n"))
      print(string("typeof(controlMatrix) = ",typeof(controlMatrix),"\n"))
    end

    # initialize problem data
    ps = PS_data(Nck=Nck,
                 Ni=Ni,
                  τ=τ,
                  ω=ω,
                 t0=t0,
                 tf=tf,
        stateMatrix=stateMatrix,
      controlMatrix=controlMatrix
                );

    nlp = NLP_data(numStates=numStates,
                        numStatePoints=numStatePoints,
                        numControls=numControls,
                        numControlPoints=numControlPoints,
                        lengthControlVector=lengthControlVector,
                        lengthStateVector=lengthStateVector,
                        lengthDecVector=lengthDecVector,
                        stateStartIdx=stateStartIdx,
                        stateStopIdx=stateStopIdx,
                        controlStartIdx=controlStartIdx,
                        controlStopIdx=controlStopIdx,
                        timeStartIdx=timeStartIdx,
                        timeStopIdx=timeStopIdx,
                        stateVector=stateVector,
                        controlVector=controlVector,
                        decisionVector=decisionVector
                   );

    return ps, nlp
end

initialize_NLP2

In [2]:
ps, nlp = initialize_NLP2(numStates=1,numControls=1);
print(ps,"\n")
print(nlp,"\n")

[5,5,5,5,5,5,5,5,5,5]typeof(Nck) = Array{Int64,1}
typeof(Ni) = Int64
typeof(τ) = Array{Array{Float64,1},1}
typeof(ω) = Array{Array{Float64,1},1}
typeof(t0) = Float64
typeof(tf) = Float64
typeof(stateMatrix) = Array{Array{Float64,2},1}
typeof(controlMatrix) = Array{Array{Float64,2},1}
NLOptControl.PS_data
  Nck: [4,4,4,4,4,4,4,4,4,4]
  Ni: 10
  τ: Array{Float64,1}[[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824],[-1.0,-0.575319,0.181066,0.822824]]
  ω: Array{Float64,1}[[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,0.776387,0.440924],[0.125,0.657689,

In [12]:
ps, nlp = initialize_NLP2(numStates=1,numControls=1,Ni=2,Nck=[3, 3]);


[4,4]typeof(Nck) = Array{Int64,1}
typeof(Ni) = Int64
typeof(τ) = Array{Array{Float64,1},1}
typeof(ω) = Array{Array{Float64,1},1}
typeof(t0) = Float64
typeof(tf) = Float64
typeof(stateMatrix) = Array{Array{Float64,2},1}
typeof(controlMatrix) = Array{Array{Float64,2},1}


In [13]:
ps

NLOptControl.PS_data
  Nck: [3,3]
  Ni: 2
  τ: Array{Float64,1}[[-1.0,-0.289898,0.689898],[-1.0,-0.289898,0.689898]]
  ω: Array{Float64,1}[[0.222222,1.02497,0.752806],[0.222222,1.02497,0.752806]]
  t0: 0.0
  tf: 0.0
  stateMatrix: Array{Float64,2}[
[0.0; 0.0; 0.0; 0.0],

[0.0; 0.0; 0.0; 0.0]]
  controlMatrix: Array{Float64,2}[
[0.0; 0.0; 0.0],

[0.0; 0.0; 0.0]]

In [14]:
nlp

NLOptControl.NLP_data
  numStates: 1
  numStatePoints: [4,4]
  lengthStateVector: 8
  numControls: 1
  numControlPoints: [3,3]
  lengthControlVector: 6
  lengthDecVector: 16
  stateStartIdx: 1
  stateStopIdx: 8
  controlStartIdx: 9
  controlStopIdx: 14
  timeStartIdx: 15
  timeStopIdx: 16
  stateVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  controlVector: [0.0,0.0,0.0,0.0,0.0,0.0]
  decisionVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

Test passing extra arguments functionality

# 

# 